In [79]:
# import library
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import csv
from subprocess import call

pdf_url = ""
pdf_filename = ""
call(["curl", pdf_url, 
    '-H', 'Connection: keep-alive', 
    '-H', 'Cache-Control: max-age=0', 
    '-H', 'Upgrade-Insecure-Requests: 1', 
    '-H', 'User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36', 
    '-H', 'Sec-Fetch-Mode: navigate', 
    '-H', 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 
    '-H', 'Sec-Fetch-Site: cross-site', 
    '-H', 'Accept-Encoding: gzip, deflate, br', 
    '-H', 'Accept-Language: en-US,en;q=0.9,bn;q=0.8', 
    '-H', 'Cookie: bbb=rd102o00000000000000000000ffff978432aao80', 
    '--compressed', '--output', pdf_filename])

import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [80]:
# get cars_dict and cars_no variable
cars_dict = {}
cars_no = 0

In [81]:
# looping for page of website
for n in range(570,605):
    # get url and html.parser of page
    url = 'https://www.mobil123.com/mobil-dijual/toyota/indonesia?type=used&page_number='+str(n)+'&page_size=25'
    page = requests.get(url)
    soup = BeautifulSoup(page.text,'html.parser')
    
    # get tag article with class = listing from url
    tables = soup.find_all("article", {'class':'listing'})
    
    # return cars_no variable to self
    cars_no = cars_no
    # looping for get feature of car
    for table in tables:
        display_title = table['data-display-title']
        line_text = table['data-default-line-text']
        brand = table['data-make']
        model = table['data-model']
        year = table['data-year']
        mileage = table['data-mileage']
        variant = table['data-variant']
        url = table['data-url']
        
        # get sub url and html.parser of sub page
        sub_url  = table['data-url']
        sub_page = requests.get(sub_url)
        sub_soup = BeautifulSoup(sub_page.text,'html.parser')
        
        # get tag section
        sub_tables = sub_soup.find('section', class_ = 'c-section c-section--key-details u-margin-ends-lg u-margin-ends-sm@mobile u-order-3@mobile')
        sub_table_location = sub_soup.find('section', class_ = 'c-section--content u-border-top u-padding-ends-md')
        # get feature warna, engine, transmisi, seats
        color = sub_tables.find_all('span', class_ = 'u-text-bold u-block')[3].text
        engine = sub_tables.find_all('span', class_ = 'u-text-bold u-block')[4].text
        transmisi = sub_tables.find_all('span', class_ = 'u-text-bold u-block')[5].text
        seats = sub_tables.find_all('span', class_ = 'u-text-bold u-block')[6].text
        province = sub_table_location.find_all('span', class_ = 'c-chip c-chip--sm u-rounded u-margin-right-xxs u-margin-bottom-xxs')[0].text
        area = sub_table_location.find_all('span', class_ = 'c-chip c-chip--sm u-rounded u-margin-right-xxs u-margin-bottom-xxs')[1].text

        # get tag section, div, span from url
        spec_section = sub_soup.find('section', class_ = 'c-section--content u-border-top u-padding-ends-md')
        spec_div = spec_section.find('div', class_ = "o-container")
        spec_div2 = spec_div.find('div', class_ = "c-card__tabs")
        spesifikasi = spec_div2.find_all('span', class_ = ('u-width-1/2', 'u-text-bold u-width-1/2 u-align-right'))

        case_list = {}
        # get feature from sub page
        for n in range(0,len(spesifikasi),2):
            case = {spesifikasi[n].text:spesifikasi[n+1].text}
            case_list.update(case)
        try:
            door = case_list['Pintu']
        except KeyError:
            print('default door')

        try:
            fuel_type = case_list['Tipe Bahan Bakar']
        except KeyError:
            print('default fuel')

        try:
            power = case_list['Tenaga Puncak (hp)']
        except KeyError:
            print('default power')
        
        # fill value to cars_dict
        cars_dict[cars_no] = [display_title, line_text, url, brand, model, variant, mileage, transmisi, power, engine, door,
                         seats, fuel_type, color, year, province, area]
        cars_no = cars_no + 1



session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

session.get(url)

default power
default power
default power
default power
default power
default power
default power
default power
default power
default power
default power
default power
default power
default power
default power
default power
default power
default power
default power


<Response [200]>

In [82]:
cars_dict_df = pd.DataFrame.from_dict(cars_dict, orient='index',
                                      columns=['display_title', 'line_text', 'url', 'brand', 'model', 'variant', \
                                                'mileage', 'transmisi', 'power', 'engine', 'door', 'seats', 'fuel_type', \
                                                'color', 'year', 'province', 'area'])

#cars_dict_df.to_csv('mobil123_toyota_full.csv')

cars_dict_df.to_csv('mobil123_toyota_full.csv', mode='a', index=True, header=False)